# **Homework 2 - Semantic Segmentation**

Objective: Implement a U-Net Network for semantic segmentation.



Dataset:

<figure>
<center>
<img src= 'https://raw.githubusercontent.com/mabelortega/DL_Semantic_Segmentation/main/Figures/drawing1.png'/>
</figure>

You must train the network model by using these images [https://drive.google.com/file/d/1TU2nTVGS2932hRs1u-ma4r3vmgqHRbMO/view?usp=sharing]. Image_Train.tif and Reference_Train.tif images and it must be evaluated on Image_Test.tif and Reference_Test.tif images. You can use this notebook that contains some basic functions.

Experimental Protocol

Load the input data
1.     Load the images provided from 2D Semantic Labeling-Vaihingen dataset using the function load_tiff_image(image) and normalize the data into the range [0,1] using the function normalization (image)

Train the FCN model
2.     To train the FCN model you need patches as input. You must extract patches of size w-by-w-by-c pixels from Image_Train and patches with size w-by-w from Reference_Train. The number of patches and the w must be chosen based on the input size of network.

3.     Split randomly the training patches into two sets: Training (80%) and validation (20%).

4.     Convert the patches of the Reference image into one-hot encoding base on the number of classes. Hint: Use the function tf.keras.utils.to_categorical.

5.     Create the function of the U-Net model - Using skip connections: Hint: use tensorflow.keras.layers.concatenate

6.     For training, use the weighted_categorical_crossentropy as a loss function. Hint: To compute the weights you must count the number of pixels of each class and apply the formula: w_i = #total_pixels / #pixels_of_class_i



7.     Train the model using Train_model() function, which has as input the training and validation patches. You must the best model adding the early stop strategy with patience equal to 10.

8.     Extract patches from the test images and test the model using Test(model, patch_test).

9.     Reconstruct the prediction (whole test image)

The report must present the classification results as label images, and report accuracy metrics (overall and average class accuracies, F1-score) you also must change the size of the extracted patches to compare the results (32x32, 64x64, 128x128)

# **Import the libraries**

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras as K
from PIL import Image
from sklearn.utils import shuffle
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score


2023-09-15 17:19:03.675159: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 17:19:03.785017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 17:19:03.786576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 17:19:06.434177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load images

In [5]:
def load_tiff_image(path, grayscale=False):
    image = Image.open(path)
    image = image.convert("L") if grayscale else image

    return np.array(image)


train_image = load_tiff_image("images/Images_Train/Image_Train.tif")
train_reference = load_tiff_image(
    "images/Images_Train/Reference_Train.tif",
    grayscale=True,
)

test_image = load_tiff_image("images/Images_Test/Image_Test.tif")
test_reference = load_tiff_image(
    "images/Images_Test/Reference_Test.tif",
    grayscale=True,
)


In [6]:
def one_hot_encode(image: np.ndarray):
    W, H = image.shape
    
    colors = np.sort(np.unique(image))
    image_encoded = np.zeros((W, H, len(colors)))

    for i, color in enumerate(colors):
        image_encoded[:, :, i] = image == color

    return image_encoded

train_reference_encoded = one_hot_encode(train_reference)
test_reference_encoded = one_hot_encode(test_reference)


In [7]:
def normalization(image: np.ndarray):
    W, H, C = image.shape

    scaler = MinMaxScaler(feature_range=(0, 1))
    image_normalized = scaler.fit_transform(image.reshape((W * H), C))
    image_normalized = image_normalized.reshape(W, H, C)

    return image_normalized

train_image_normalized = normalization(train_image)
test_image_normalized = normalization(test_image)


# **Define the functions**

In [8]:

def extract_patches(image, size, stride):
    W, H, _ = image.shape
    patches = []

    for i in range(0, W, stride):
        for j in range(0, H, stride):
            if i + size > W or j + size > H:
                continue

            patch = image[i : i + size, j : j + size]
            patches.append(patch)

    return np.array(patches).reshape(-1, size, size, 3)


def unet(input_shape, n_classes):
    pass
    # input_img = Input(input_shape)
    # # You must complete the U-Net architecture
    # output = Conv2D(n_classes, (1, 1), activation="softmax")(merged)
    # return Model(inputs=input_img, outputs=output, name="U-Net")


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred)
        loss = loss * weights
        loss = -K.mean(loss, -1)
        return loss

    return loss



In [ ]:

def train_model(
    net,
    patches_train,
    patches_tr_lb_h,
    patches_val,
    patches_val_lb_h,
    batch_size,
    epochs,
):
    print("Start training.. ")
    for epoch in range(epochs):
        loss_tr = np.zeros((1, 2))
        loss_val = np.zeros((1, 2))
        # Computing the number of batchs
        n_batchs_tr = patches_train.shape[0] // batch_size
        # Random shuffle the data
        patches_train, patches_tr_lb_h = shuffle(
            patches_train, patches_tr_lb_h, random_state=0
        )

        # Training the network per batch
        for batch in range(n_batchs_tr):
            x_train_b = patches_train[
                batch * batch_size : (batch + 1) * batch_size, :, :, :
            ]
            y_train_h_b = patches_tr_lb_h[
                batch * batch_size : (batch + 1) * batch_size, :, :, :
            ]
            loss_tr = loss_tr + net.train_on_batch(x_train_b, y_train_h_b)

        # Training loss
        loss_tr = loss_tr / n_batchs_tr
        print(
            "%d [Training loss: %f , Train acc.: %.2f%%]"
            % (epoch, loss_tr[0, 0], 100 * loss_tr[0, 1])
        )

        # Computing the number of batchs
        n_batchs_val = patches_val.shape[0] // batch_size

        # Evaluating the model in the validation set
        for batch in range(n_batchs_val):
            x_val_b = patches_val[
                batch * batch_size : (batch + 1) * batch_size, :, :, :
            ]
            y_val_h_b = patches_val_lb_h[
                batch * batch_size : (batch + 1) * batch_size, :, :, :
            ]
            loss_val = loss_val + net.test_on_batch(x_val_b, y_val_h_b)

        # validation loss
        loss_val = loss_val / n_batchs_val
        print(
            "%d [Validation loss: %f , Validation acc.: %.2f%%]"
            % (epoch, loss_val[0, 0], 100 * loss_val[0, 1])
        )
        # Add early stopping


def test(model, patch_test):
    result = model.predict(patch_test)
    predicted_class = np.argmax(result, axis=-1)
    return predicted_class


def compute_metrics(true_labels, predicted_labels):
    accuracy = 100 * accuracy_score(true_labels, predicted_labels)
    f1score = 100 * f1_score(true_labels, predicted_labels, average=None)
    recall = 100 * recall_score(true_labels, predicted_labels, average=None)
    precision = 100 * precision_score(true_labels, predicted_labels, average=None)
    return accuracy, f1score, recall, precision


In [ ]:
# # Extract training patches
# patches_train, patches_train_ref = extract_patches(tr_img, tr_ref_img, patch_size, stride)

# # One hot encoding
# patches_tr_lb_h

# # Train the model
# adam = Adam(lr = 0.0001 , beta_1=0.9)
# net = unet((patch_size, patch_size, channels), number_class)
# loss = weighted_categorical_crossentropy(weights)
# net.summary()
# net.compile(loss = loss, optimizer=adam , metrics=['accuracy'])

# # load the model
# model = load_model(name)

# # Test the model
# predicted_labels = Test(model, patch_test)

# # Metrics
# compute_metrics(true_labels, predicted_labels)

# # Plot the prediction (whole test image)


NameError: ignored